# 🤖 Interactive FinOps Analysis with Gemini 2.5 Pro
\nThis notebook provides a truly interactive experience for analyzing your cloud cost data using Gemini 2.5 Pro. Use the widgets below to ask your own questions and leverage advanced features like Code Execution and URL Context.

## ⚠️ Important: Setup and Authentication
\nEnsure you have run `bash scripts/setup_gcp_notebook.sh` and are authenticated to Google Cloud (`gcloud auth application-default login`).

## 📚 Step 1: Configuration, Imports, and Initialization

In [ ]:
# @markdown Specify the path to your configuration file.\nCONFIG_PATH = 'config.yaml' # @param {type:"string"}\n\nimport logging\nimport pandas as pd\nimport ipywidgets as widgets\nfrom IPython.display import Markdown, display, clear_output\nfrom finops_analysis_platform.config_manager import ConfigManager\nfrom finops_analysis_platform.data_loader import load_data_from_config\nfrom finops_analysis_platform.gemini_service import initialize_gemini, generate_content\nfrom finops_analysis_platform.core import CUDAnalyzer\nfrom google.genai.types import Tool, ToolCodeExecution, UrlContext\n\nlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')\n\nconfig_manager = ConfigManager(config_path=CONFIG_PATH)\nproject_id = config_manager.get('gcp.project_id')\nlocation = config_manager.get('gcp.location', 'us-central1')\n\nclient = initialize_gemini(project_id=project_id, location=location)\nmodel_id = 'gemini-2.5-pro'\n\nprint(f"✅ Gemini client initialized for project '{project_id}'.")

## 📂 Step 2: Load Data

In [ ]:
data = load_data_from_config(config_manager)\nbilling_df = data.get('billing')\nrecommendations_df = data.get('recommendations')\n\nprint("✅ Data loaded successfully. The billing data is available in the `billing_df` DataFrame.")

## 💻 Step 3: Interactive Analysis with Code Execution
\nAsk a question about your billing data. Gemini will write and execute code to answer it. **Reference the data as `billing_df`.**

In [ ]:
code_prompt_input = widgets.Textarea(\n    value='Analyze the billing data in `billing_df`. What are the top 3 services by cost? Display the result as a table.',\n    placeholder='Type your prompt here... e.g., Show me the total cost per project.',\n    description='Prompt:',\n    layout={'width': '100%', 'height': '100px'}\n)\ncode_run_button = widgets.Button(description="Run Analysis")\ncode_output_area = widgets.Output()\n\ndef run_code_execution(b):\n    with code_output_area:\n        clear_output()\n        print("Generating and executing code...")\n        code_execution_tool = Tool(code_execution=ToolCodeExecution())\n        response = generate_content(client, model_id, code_prompt_input.value, tools=[code_execution_tool])\n        clear_output()\n        if response:\n            for part in response.candidates[0].content.parts:\n                if part.executable_code:\n                    display(Markdown(f"**Generated Code:**\n```python\n{part.executable_code.code}\n```"))\n                if part.code_execution_result:\n                    display(Markdown(f"**Execution Result:**\n```\n{part.code_execution_result.output}\n```"))\n        else:\n            print("Failed to get a response from the API.")\n\ncode_run_button.on_click(run_code_execution)\ndisplay(code_prompt_input, code_run_button, code_output_area)

## 🔗 Step 4: Interactive Analysis with URL Context
\nAsk a question based on the content of a public URL.

In [ ]:
url_input = widgets.Text(value='https://cloud.google.com/compute/all-pricing', description='URL:')\nurl_prompt_input = widgets.Textarea(\n    value='Summarize the key differences in pricing between N2, E2, and C2 machine types.',\n    placeholder='Type your question about the URL content... e.g., What is the main point of this article?',\n    description='Prompt:',\n    layout={'width': '100%', 'height': '80px'}\n)\nurl_run_button = widgets.Button(description="Run Analysis")\nurl_output_area = widgets.Output()\n\ndef run_url_analysis(b):\n    with url_output_area:\n        clear_output()\n        print("Analyzing URL content...")\n        url_context_tool = Tool(url_context=UrlContext())\n        prompt = f"Based on this URL: {url_input.value}, {url_prompt_input.value}"\n        response = generate_content(client, model_id, prompt, tools=[url_context_tool])\n        clear_output()\n        if response and response.text:\n            display(Markdown(response.text))\n        else:\n            print("Failed to get a response from the API.")\n\nurl_run_button.on_click(run_url_analysis)\ndisplay(url_input, url_prompt_input, url_run_button, url_output_area)

## 💎 Step 5: Advanced AI Portfolio Optimization
\nLeverage Gemini's analytical capabilities to get a sophisticated, risk-adjusted CUD portfolio recommendation. This uses the new `generate_cud_portfolio_optimization` feature.

In [ ]:
portfolio_button = widgets.Button(description="Generate AI Portfolio Recommendation")\nportfolio_output_area = widgets.Output()\n\ndef run_portfolio_optimization(b):\n    with portfolio_output_area:\n        clear_output()\n        print("Running advanced portfolio analysis...")\n        analyzer = CUDAnalyzer(config_manager=config_manager, billing_data=billing_df)\n        # We need the savings_by_machine data first\n        machine_distribution = analyzer._analyze_machine_distribution()\n        savings_by_machine = analyzer._calculate_savings_by_machine(machine_distribution)\n        ai_portfolio = analyzer.generate_cud_portfolio_optimization(savings_by_machine)\n        clear_output()\n        if ai_portfolio and 'error' not in ai_portfolio:\n            display(Markdown(f"### AI-Powered Portfolio Strategy\n**Summary:** {ai_portfolio.get('strategy_summary', 'N/A')}"))\n            display(pd.DataFrame(ai_portfolio.get('portfolio', [])))\n        else:\n            display(Markdown(f"**Could not generate AI portfolio:**\n`{ai_portfolio}`"))\n\nportfolio_button.on_click(run_portfolio_optimization)\ndisplay(portfolio_button, portfolio_output_area)